In [51]:
%run ../utils.ipynb
from bs4 import BeautifulSoup
import requests
import sys
import time
import pandas as pd
import numpy as np
import urllib.robotparser as urobot
from tqdm import tqdm_notebook as tqdm
import validators
import os
import threading
import logging

header = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_5) AppleWebKit/605.1.15 (KHTML, like Gecko) Safari/605.1.15'}

urls = [
        "https://www.magazineluiza.com.br",
        "https://www.colombo.com.br",
        "https://www.amazon.com.br",
        "https://www.taqi.com.br",
        "https://www.kabum.com.br",
        "https://www.ibyte.com.br",
        "https://www.cissamagazine.com.br",
        "https://www.promobit.com.br",
        "https://www.havan.com.br",
        "https://www.avenida.com.br"]

#urls = ["https://www.avenida.com.br"]

In [52]:
#Class definition of the heuristic_crawler
class heuristic_crawler:
    def __init__(self, url):
        self.url = url                
        self.robotParser = getRobot(url)           #robot parser to check if a link is valid or not
        self.links_list = []                       #list of allowed links
        self.invalid_links = []                    #list of invalid links
        self.file_name  = set_file_name(url)       #name of the files generate by the crawler 

    def get_links(self):
        actual_link = self.url
        link_count = 0
        index = 0
        number_of_links = 1000
        visited_links = [self.url]
        
        while (len(self.links_list) < number_of_links):
            try:
                req = requests.get(actual_link, headers=header)
                if(req.status_code == 200):    
                    soup = BeautifulSoup(req.text)
                    pageLinks = soup.findAll("a", href=True)
                    
                    for a in pageLinks:
                        #formata o link na tentativa de obter um link válido
                        link = format_link(self.url, a["href"])
                        #print(link)
                        #verifica se o link é valido
                        if(not validators.url(link)):
                            if(link not in self.invalid_links):
                                self.invalid_links.append(link)
                            
                        #adiciona na lista de links do crawler caso seja um link não visitado, válido e que o robots.txt permita
                        elif((self.robotParser.can_fetch("*", link)) and (link not in visited_links) and (self.heuristic_check(link))):
                            #print("{} - {}".format(link_count, link))
                            self.links_list.append(link)
                            visited_links.append(link)
                            link_count += 1
                        
                        
            except Exception:
                if(link not in self.invalid_links):
                    self.invalid_links.append(link)
            finally: 
                if(index >= len(self.links_list) or len(self.links_list) >= number_of_links):
                    #print("****END: {}****".format(self.url))
                    return
                actual_link = self.links_list[index]
                #print("Link Atual: {}".format(actual_link))
                index += 1
                time.sleep(1)
                
    #saves the links as csv 
    def save_as_csv(self):
        folders = ["links", "invalid_links"]
        for folder in folders:
            if not os.path.exists(folder):
                os.makedirs(folder)
        
        #saves valid links
        ID = np.arange(len(self.links_list))
        dictionary = {'id' : ID, 'links' : self.links_list}
        df = pd.DataFrame(dictionary)
        df.to_csv(('links/' + self.file_name + '.csv'),header=True, index=False, encoding='utf-8')
        
        #saves invalid links
        ID = np.arange(len(self.invalid_links))
        dictionary = {'id' : ID, 'links' : self.invalid_links}
        df = pd.DataFrame(dictionary)
        df.to_csv(('invalid_links/' + self.file_name + '.csv'),header=True, index=False, encoding='utf-8')
    
    #save the robots from a url
    def save_robot(self):
        robot_url = self.url + "/robots.txt"
        req = requests.get(robot_url,headers=header)
        content = req.text
        save_file(content, "robots", (self.file_name + ".txt"))
    
    #verifies if a link passes in the coditions of the heuristic
    def heuristic_check(self, link):
        KEYWORDS = ["Celular", "celular", "Smartphone", "smartphone", "iphone", "Telefon", "telefon"]
        BADWORDS  = ["televenda", "Televenda", "sac", "carr", "javascript", "acessorio", "Acessorio"]
        result = False
        
        for word in KEYWORDS:
            if(word in link):
                result = True
                for bad_word in BADWORDS:
                    if(bad_word in link):
                        result = False
        return result
    
    #getters and setters methods
    def get_url(self):
        return self.url
    def set_url(self, url):
        self.url = url
    def get_links_list(self):
        return self.links_list
    def set_links_list(self, links):
        self.links_list = links

In [53]:
#Saves the links from each site
def save_links(crawler):
    crawler = heuristic_crawler(url)
    crawler.save_robot()
    crawler.get_links()
    crawler.save_as_csv()

# Parallel saves the links in .csv 

In [29]:
#configures logger to track progress of the threads
logger = logging.getLogger()
logger.setLevel(logging.INFO)

threads = []
for url in urls:
    thread_name = url.split(".")[1]
    crawler = heuristic_crawler(url)
    thread = threading.Thread(target=save_links, args=(crawler,), name=thread_name)
    threads.append(thread)
    print("Starts crawler on: {}".format(url))
    thread.start()
    time.sleep(0.2)
    
for thread in tqdm(threads):
    thread.join()
    logging.info("finished thread: {}".format(thread.name))

Starts crawler on: https://www.magazineluiza.com.br
Starts crawler on: https://www.colombo.com.br
Starts crawler on: https://www.taqi.com.br
Starts crawler on: https://www.kabum.com.br
Starts crawler on: https://www.ibyte.com.br
Starts crawler on: https://www.cissamagazine.com.br
Starts crawler on: https://www.promobit.com.br
Starts crawler on: https://www.havan.com.br
Starts crawler on: https://www.avenida.com.br


****END: https://www.cissamagazine.com.br****


INFO:root:finished thread: magazineluiza


****END: https://www.magazineluiza.com.br****
****END: https://www.avenida.com.br****
****END: https://www.ibyte.com.br****
****END: https://www.havan.com.br****


INFO:root:finished thread: colombo


****END: https://www.colombo.com.br****
****END: https://www.promobit.com.br****


INFO:root:finished thread: taqi


****END: https://www.taqi.com.br****


INFO:root:finished thread: kabum
INFO:root:finished thread: ibyte
INFO:root:finished thread: cissamagazine
INFO:root:finished thread: promobit
INFO:root:finished thread: havan
INFO:root:finished thread: avenida


****END: https://www.kabum.com.br****



# Saves the links from one site

In [24]:
url = urls[2]
crawler = heuristic_crawler(url)
print("Starts crawler on: {}".format(url))
save_links(url)

Starts crawler on: https://www.amazon.com.br
END: https://www.amazon.com.br
Finished crawler on: https://www.amazon.com.br
